In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.dpi': 150})
plt.rcParams['font.sans-serif'] = 'Times New Roman'
import seaborn as sns
import plotly.express as px
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import matplotlib
import re
import ast
import os

In [12]:
data = pd.read_csv('/Users/ainsleylewis/Documents/Astronomy/arXiver/2025_Data_missing.csv')

In [13]:
data['num_checks'] = 0

In [14]:
# Cross-check pages and figures from comments
# Percentage of mismatches
total_entries = len(data)
mismatches = 0
for i in range(len(data)):
    if isinstance(data.at[i, 'comments'], str):
        if 'pages' in data.at[i, 'comments'].lower():
            try:
                pages_part = data.at[i, 'comments'].lower().split('pages')[0]
                pages_str = pages_part.split()[-1]
                pages_str = pages_str.replace(',', '').replace('.', '').replace('(', '').replace(')', '')
                pages_str = int(pages_str)
                predicted_pages = data.at[i, 'pages']
                if int(pages_str) != predicted_pages:
                    mismatches += 1
            except:
                pass
        if 'figures' in data.at[i, 'comments'].lower():
            try:
                figures_part = data.at[i, 'comments'].lower().split('figures')[0]
                figures_str = figures_part.split()[-1]
                figures_str = figures_str.replace(',', '').replace('.', '').replace('(', '').replace(')', '')
                figures_str = int(figures_str)
                predicted_figures = data.at[i, 'figures']
                if int(figures_str) != predicted_figures:
                    mismatches += 1
            except:
                pass
        if 'tables' in data.at[i, 'comments'].lower():
            try:
                tables_part = data.at[i, 'comments'].lower().split('tables')[0]
                tables_str = tables_part.split()[-1]
                tables_str = tables_str.replace(',', '').replace('.', '').replace('(', '').replace(')', '')
                tables_str = int(tables_str)
                predicted_tables = data.at[i, 'tables']
                if int(tables_str) != predicted_tables:
                    mismatches += 1
            except:
                pass

mismatch_percentage = (mismatches / total_entries) * 100
print(f"Total entries: {total_entries}")
print(f"Mismatches found: {mismatches}")
print(f"Mismatch percentage: {mismatch_percentage:.2f}%")

Total entries: 327
Mismatches found: 7
Mismatch percentage: 2.14%


In [15]:
# Un-naturally large page or figure counts
# Percemtage of entries with large counts
total_entries = len(data)
large_counts = 0
for i in range(len(data)):
    if data.at[i, 'pages'] > 100:
        if data.at[i, 'num_checks'] == 0:
            print(f"Un-naturally large page count for paper {data['pdf_link'][i]}: {data.at[i, 'pages']} pages")
            title = data.at[i, 'title']
            corrected_pages = input(str(title) + " : Current index " + str(i) + " ")
            if corrected_pages.isdigit():
                data.at[i, 'pages'] = int(corrected_pages)
                large_counts += 1
                data.at[i, 'num_checks'] += 1
    if data.at[i, 'figures'] > 50:
        if data.at[i, 'num_checks'] == 0:
            print(f"Un-naturally large figure count at index {data['pdf_link'][i]}: {data.at[i, 'figures']} figures")
            title = data.at[i, 'title']
            corrected_figures = input(str(title) + " : Current index " + str(i) + " ")
            if corrected_figures.isdigit():
                data.at[i, 'figures'] = int(corrected_figures)
                large_counts += 1
                data.at[i, 'num_checks'] += 1
    if data.at[i, 'tables'] > 20:
        if data.at[i, 'num_checks'] == 0:
            print(f"Un-naturally large table count at index {data['pdf_link'][i]}: {data.at[i, 'tables']} tables")
            title = data.at[i, 'title']
            corrected_tables = input(str(title) + " : Current index " + str(i) + " ")
            if corrected_tables.isdigit():
                data.at[i, 'tables'] = int(corrected_tables)
                large_counts += 1
                data.at[i, 'num_checks'] += 1

large_counts_percentage = (large_counts / total_entries) * 100
print(f"Total entries: {total_entries}")
print(f"Entries with large counts corrected: {large_counts}")

Un-naturally large figure count at index arxiv.org/pdf/2512.20716: 58.0 figures
Un-naturally large figure count at index arxiv.org/pdf/2512.22399: 435.0 figures
Un-naturally large figure count at index arxiv.org/pdf/2512.23267: 128.0 figures
Un-naturally large figure count at index arxiv.org/pdf/2512.23815: 87.0 figures
Un-naturally large page count for paper arxiv.org/pdf/2512.24350: 195.0 pages
Un-naturally large page count for paper arxiv.org/pdf/2512.24465: 188.0 pages
Total entries: 327
Entries with large counts corrected: 4


In [16]:
# Count the number of times the words 'submitted' or 'journal' or 'published' or 'accepted' appears in comments
counter = 0
for i in range(len(data)):
    if data['published_journal'][i] is not None and isinstance(data['published_journal'][i], str):
        continue
    elif isinstance(data.at[i, 'comments'], str):
        comments_lower = data.at[i, 'comments'].lower()
        if 'published' in comments_lower or 'accepted' in comments_lower:
            print(f"Publication info found in comments for index {i}: {data.at[i, 'comments']}")
            counter += 1

print(f"Total papers with publication info in comments: {counter}")

Publication info found in comments for index 7: 19 pages, 14 figures, 3 tables; Accepted for publication in A&A
Publication info found in comments for index 14: Accepted for publication in MNRASL. 5 pages. 5 figures
Publication info found in comments for index 16: 26 pages, 14 figures, 5 tables. To be published in PASP
Publication info found in comments for index 17: Accepted by ApjL
Publication info found in comments for index 22: 13 pages, 6 figures, 2 tables. Accepted for publication in The Astrophysical Journal
Publication info found in comments for index 31: Accepted for publication in Astronomy & Astrophysics
Publication info found in comments for index 33: Accepted for publication in The Astrophysical Journal (ApJ). 19 pages, 6 figures
Publication info found in comments for index 35: 17 pages, 14 figures, 3 tables. Accepted for publication in MNRAS. Supplementary figures available at this https URL . Code available at this https URL
Publication info found in comments for index 3

In [18]:
data.to_csv('/Users/ainsleylewis/Documents/Astronomy/arXiver/2025_Data_missing.csv', index=False)

In [19]:
# Adding in the citation info
authors = pd.read_csv('/Users/ainsleylewis/Documents/Astronomy/arXiver/papers_with_ads_metrics_7.csv')
print(authors['affiliations'].notna().sum())

13


In [22]:

def normalize_name(name):
    if not isinstance(name, str):
        return ""
    return name.replace('-', '').replace(' ', '').lower()

def extract_affiliations_list(row):
    authors_str = row['authors']
    affil_str = row['affiliations']
    
    try:
        authors = ast.literal_eval(authors_str) if isinstance(authors_str, str) else []
    except:
        authors = []
        
    if not authors:
        return []
    
    if pd.isna(affil_str):
        return [None] * len(authors)
        
    # Split by Name: (no colons or semicolons allowed in name)
    parts = re.split(r'(?:^|;|;;)\s*([^:;]+):', affil_str)
    
    affil_map = {}
    
    for i in range(1, len(parts), 2):
        name_key = parts[i].strip()
        affil_val = parts[i+1].strip()
        affil_val = re.sub(r'[;]+$', '', affil_val).strip()
        
        # 1. Map original name
        affil_map[name_key] = affil_val
        
        # 2. Map normalized name
        affil_map[normalize_name(name_key)] = affil_val
        
        # 3. Handle "Last, First" -> "First Last" mapping
        if ',' in name_key:
            last, first = name_key.split(',', 1)
            first_last = f"{first.strip()} {last.strip()}"
            affil_map[first_last] = affil_val
            affil_map[normalize_name(first_last)] = affil_val
            
            # 4. Handle 'ü' -> 'v' substitution (common in Chinese names transliteration)
            if 'ü' in name_key:
                name_v = name_key.replace('ü', 'v')
                first_last_v = first_last.replace('ü', 'v')
                affil_map[normalize_name(name_v)] = affil_val
                affil_map[normalize_name(first_last_v)] = affil_val
                
    result = []
    for author in authors:
        # Try looking up by exact match
        affil = affil_map.get(author)
        
        # Try looking up by normalized match
        if affil is None:
            affil = affil_map.get(normalize_name(author))
            
        # Try constructing Last, First key manually from Author string if not found
        if affil is None:
            parts_name = author.split()
            if len(parts_name) > 1:
                # "First Middle Last" -> "Last, First Middle"
                last_first = f"{parts_name[-1]}, {' '.join(parts_name[:-1])}"
                affil = affil_map.get(last_first)
                if affil is None:
                    affil = affil_map.get(normalize_name(last_first))
                    
        # Handle ü -> v check for author name input too
        if affil is None and 'v' in author:
             # e.g. Author is "Lv", Key might be "Lü"
             # This direction is harder to map solely via map keys unless we permute author string
             # But we enriched the map with 'v' versions of 'ü' keys.
             # If the author has 'v', and key has 'v', it should match via normalized map.
             # What if Author has 'Lv' and key has 'Lv' (already matched)?
             pass
        
        result.append(affil)
        
    return result

# Add affiliation columns to data
merged_data = data.merge(authors[['title', 'affiliations', 'total_citations', 'non_self_citations']], 
    on='title', 
    how='left'
)

merged_data['affiliations'] = merged_data.apply(extract_affiliations_list, axis=1)
merged_data[['authors', 'affiliations']].head(10)

,authors,affiliations
0,['John K. Nino'],[None]
1,"['Matthew VanDyke', 'Kexuan Wu', 'Sukanta Bose']","[None, None, None]"
2,"['Akke Corporaal', 'Toon De Prins', 'Léa Planq...","[None, None, None, None, None, None, None, Non..."
3,"['Yutaka Fujita', 'Rohta Takahashi', 'Norita K...","[None, None, None]"
4,"['Matteo Cantiello', 'Jake B. Hassan', 'Rosalb...","[None, None, None, None, None, None, None, None]"
5,"['Maria Ramos', 'Timothy Cohen', 'Mariangela L...","[None, None, None]"
6,"['M. Grant Roberts', 'Aarna Garg', 'Tesla Jelt...","[None, None, None, None]"
7,"['A. Damonte', 'I. Pillitteri', 'A. Maggio', '...","[None, None, None, None, None]"
8,"['Alison L. Coil', 'David S. N. Rupke', 'Seren...","[None, None, None, None, None]"
9,['Joyful E. Mdhluli'],[None]


In [24]:
merged_data

,title,abstract,authors,figures,pages,tables,pdf_link,primary_subject,secondary_subjects,submitted_journal,published_journal,keywords,date,comments,journals,num_checks,affiliations,total_citations,non_self_citations
0,A Correlation Between Black Hole Mass and Dark...,We report the discovery of a positive correlat...,['John K. Nino'],1.0,12.0,2.0,arxiv.org/pdf/2512.17964,Astrophysics of Galaxies,['Cosmology and Nongalactic Astrophysics'],NaN,NaN,and total stellar mass—has been established fo...,2025-12-23,"6 pages, 1 figure, 2 tables. v2: Corrected Li ...",NaN,0,[None],0,0
1,CoBiTS: Single-detector discrimination of bina...,"We develop a Conformer neural network, called ...","['Matthew VanDyke', 'Kexuan Wu', 'Sukanta Bose']",13.0,22.0,0.0,arxiv.org/pdf/2512.17975,Instrumentation and Methods for Astrophysics,"['High Energy Astrophysical Phenomena', 'Gener...",NaN,NaN,[],2025-12-23,"22 pages, 13 figures",NaN,0,"[None, None, None]",0,0
2,Bridging stellar evolution and planet formatio...,"Stars and planets form, live, and evolve in un...","['Akke Corporaal', 'Toon De Prins', 'Léa Planq...",0.0,4.0,0.0,arxiv.org/pdf/2512.17976,Instrumentation and Methods for Astrophysics,"['Earth and Planetary Astrophysics', 'Solar an...",NaN,NaN,[],2025-12-23,Science white paper submitted in the context o...,NaN,0,"[None, None, None, None, None, None, None, Non...",0,0
3,Diffusive or Ballistic? Distributions and Spec...,In the standard Galactic cosmic-ray (CR) parad...,"['Yutaka Fujita', 'Rohta Takahashi', 'Norita K...",3.0,6.0,0.0,arxiv.org/pdf/2512.17994,High Energy Astrophysical Phenomena,NaN,NaN,NaN,['High energy astrophysics (739) — Cosmic rays...,2025-12-23,"6 pages, 3 figures",NaN,0,"[None, None, None]",0,0
4,Pulsational Instability of Quasi-Stars: Interp...,"The JWST discovery of ""Little Red Dots"" (LRDs)...","['Matteo Cantiello', 'Jake B. Hassan', 'Rosalb...",6.0,11.0,0.0,arxiv.org/pdf/2512.17997,High Energy Astrophysical Phenomena,"['Astrophysics of Galaxies', 'Solar and Stella...",NaN,NaN,['Early Universe — Supermassive black holes — ...,2025-12-23,"11 pages, 6 figures, submitted to ApJL",NaN,0,"[None, None, None, None, None, None, None, None]",0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,Cosmic Himalayas in CROCODILE : Probing the Ex...,The recently reported Cosmic Himalayas (CH) --...,"['Yuto Kuwayama', 'Yongming Liang', 'Kentaro N...",10.0,18.0,5.0,arxiv.org/pdf/2512.24966,Cosmology and Nongalactic Astrophysics,['Astrophysics of Galaxies'],NaN,NaN,['Hydrodynamicalsimulations(767)'],2025-12-31,"18 pages, 10 figures, submitted to ApJ",NaN,0,"[None, None, None, None, None, None, None, Non...",0,0
323,Multi-Frequency Study of FRB20201124A with the...,We present results from multi-epoch observatio...,"['C. Dudeja', 'J. Roy', 'U. Panda', 'S. Bhatta...",11.0,17.0,1.0,arxiv.org/pdf/2512.24978,High Energy Astrophysical Phenomena,NaN,NaN,NaN,[],2025-12-31,NaN,NaN,0,"[None, None, None, None]",0,0
324,Distributions of wide binary stars in theory a...,"Using the censored catalog of 103,169 resolved...",['Valeri V. Makarov'],7.0,13.0,1.0,arxiv.org/pdf/2512.25002,Solar and Stellar Astrophysics,NaN,NaN,NaN,[],2025-12-31,To be published in Astron J,NaN,0,[None],0,0
325,Towards precision cosmology with Voids x CMB c...,We construct and validate a set of multi-purpo...,"['Mar Pérez Sar', 'Carlos Hernández Monteagudo...",8.0,19.0,1.0,arxiv.org/pdf/2512.25040,Cosmology and Nongalactic Astrophysics,NaN,NaN,NaN,['Cosmology: Large-scale structure - Cosmic mi...,2025-12-31,"19 pages, 8 figures, submitted to A&A",NaN,0,"[None, None, None, None]",0,0


In [25]:
# Published Journal Flagging
merged_data['journal_flag'] = merged_data['published_journal'].notnull().astype(int)
merged_data

# Update journal_flag based on comments
for i in range(len(merged_data)):
    if merged_data['journal_flag'][i] == 1:
        continue
    elif isinstance(merged_data.at[i, 'comments'], str):
        comments_lower = merged_data.at[i, 'comments'].lower()
        if 'published' in comments_lower or 'accepted' in comments_lower:
            merged_data.at[i, 'journal_flag'] = 1

# Update journal_flag based on journals column from submitted_journals.py
for i in range(len(merged_data)):
    if merged_data['journal_flag'][i] == 1:
        continue
    elif isinstance(merged_data.at[i, 'journals'], str):
        merged_data.at[i, 'journal_flag'] = 1

# # Final published journal
# merged_data['published_journal_final'] = merged_data.apply(lambda row: row['published_journal'] if pd.notnull(row['published_journal']) else (row['journals'] if pd.notnull(row['journals']) else None), axis=1)
# merged_data

In [26]:
merged_data

,title,abstract,authors,figures,pages,tables,pdf_link,primary_subject,secondary_subjects,submitted_journal,published_journal,keywords,date,comments,journals,num_checks,affiliations,total_citations,non_self_citations,journal_flag
0,A Correlation Between Black Hole Mass and Dark...,We report the discovery of a positive correlat...,['John K. Nino'],1.0,12.0,2.0,arxiv.org/pdf/2512.17964,Astrophysics of Galaxies,['Cosmology and Nongalactic Astrophysics'],NaN,NaN,and total stellar mass—has been established fo...,2025-12-23,"6 pages, 1 figure, 2 tables. v2: Corrected Li ...",NaN,0,[None],0,0,0
1,CoBiTS: Single-detector discrimination of bina...,"We develop a Conformer neural network, called ...","['Matthew VanDyke', 'Kexuan Wu', 'Sukanta Bose']",13.0,22.0,0.0,arxiv.org/pdf/2512.17975,Instrumentation and Methods for Astrophysics,"['High Energy Astrophysical Phenomena', 'Gener...",NaN,NaN,[],2025-12-23,"22 pages, 13 figures",NaN,0,"[None, None, None]",0,0,0
2,Bridging stellar evolution and planet formatio...,"Stars and planets form, live, and evolve in un...","['Akke Corporaal', 'Toon De Prins', 'Léa Planq...",0.0,4.0,0.0,arxiv.org/pdf/2512.17976,Instrumentation and Methods for Astrophysics,"['Earth and Planetary Astrophysics', 'Solar an...",NaN,NaN,[],2025-12-23,Science white paper submitted in the context o...,NaN,0,"[None, None, None, None, None, None, None, Non...",0,0,0
3,Diffusive or Ballistic? Distributions and Spec...,In the standard Galactic cosmic-ray (CR) parad...,"['Yutaka Fujita', 'Rohta Takahashi', 'Norita K...",3.0,6.0,0.0,arxiv.org/pdf/2512.17994,High Energy Astrophysical Phenomena,NaN,NaN,NaN,['High energy astrophysics (739) — Cosmic rays...,2025-12-23,"6 pages, 3 figures",NaN,0,"[None, None, None]",0,0,0
4,Pulsational Instability of Quasi-Stars: Interp...,"The JWST discovery of ""Little Red Dots"" (LRDs)...","['Matteo Cantiello', 'Jake B. Hassan', 'Rosalb...",6.0,11.0,0.0,arxiv.org/pdf/2512.17997,High Energy Astrophysical Phenomena,"['Astrophysics of Galaxies', 'Solar and Stella...",NaN,NaN,['Early Universe — Supermassive black holes — ...,2025-12-23,"11 pages, 6 figures, submitted to ApJL",NaN,0,"[None, None, None, None, None, None, None, None]",0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,Cosmic Himalayas in CROCODILE : Probing the Ex...,The recently reported Cosmic Himalayas (CH) --...,"['Yuto Kuwayama', 'Yongming Liang', 'Kentaro N...",10.0,18.0,5.0,arxiv.org/pdf/2512.24966,Cosmology and Nongalactic Astrophysics,['Astrophysics of Galaxies'],NaN,NaN,['Hydrodynamicalsimulations(767)'],2025-12-31,"18 pages, 10 figures, submitted to ApJ",NaN,0,"[None, None, None, None, None, None, None, Non...",0,0,0
323,Multi-Frequency Study of FRB20201124A with the...,We present results from multi-epoch observatio...,"['C. Dudeja', 'J. Roy', 'U. Panda', 'S. Bhatta...",11.0,17.0,1.0,arxiv.org/pdf/2512.24978,High Energy Astrophysical Phenomena,NaN,NaN,NaN,[],2025-12-31,NaN,NaN,0,"[None, None, None, None]",0,0,0
324,Distributions of wide binary stars in theory a...,"Using the censored catalog of 103,169 resolved...",['Valeri V. Makarov'],7.0,13.0,1.0,arxiv.org/pdf/2512.25002,Solar and Stellar Astrophysics,NaN,NaN,NaN,[],2025-12-31,To be published in Astron J,NaN,0,[None],0,0,1
325,Towards precision cosmology with Voids x CMB c...,We construct and validate a set of multi-purpo...,"['Mar Pérez Sar', 'Carlos Hernández Monteagudo...",8.0,19.0,1.0,arxiv.org/pdf/2512.25040,Cosmology and Nongalactic Astrophysics,NaN,NaN,NaN,['Cosmology: Large-scale structure - Cosmic mi...,2025-12-31,"19 pages, 8 figures, submitted to A&A",NaN,0,"[None, None, None, None]",0,0,0


In [27]:
merged_data.to_csv('/Users/ainsleylewis/Documents/Astronomy/arXiver/2025_Data_missing.csv', index=False)

In [28]:
merged_data

,title,abstract,authors,figures,pages,tables,pdf_link,primary_subject,secondary_subjects,submitted_journal,published_journal,keywords,date,comments,journals,num_checks,affiliations,total_citations,non_self_citations,journal_flag
0,A Correlation Between Black Hole Mass and Dark...,We report the discovery of a positive correlat...,['John K. Nino'],1.0,12.0,2.0,arxiv.org/pdf/2512.17964,Astrophysics of Galaxies,['Cosmology and Nongalactic Astrophysics'],NaN,NaN,and total stellar mass—has been established fo...,2025-12-23,"6 pages, 1 figure, 2 tables. v2: Corrected Li ...",NaN,0,[None],0,0,0
1,CoBiTS: Single-detector discrimination of bina...,"We develop a Conformer neural network, called ...","['Matthew VanDyke', 'Kexuan Wu', 'Sukanta Bose']",13.0,22.0,0.0,arxiv.org/pdf/2512.17975,Instrumentation and Methods for Astrophysics,"['High Energy Astrophysical Phenomena', 'Gener...",NaN,NaN,[],2025-12-23,"22 pages, 13 figures",NaN,0,"[None, None, None]",0,0,0
2,Bridging stellar evolution and planet formatio...,"Stars and planets form, live, and evolve in un...","['Akke Corporaal', 'Toon De Prins', 'Léa Planq...",0.0,4.0,0.0,arxiv.org/pdf/2512.17976,Instrumentation and Methods for Astrophysics,"['Earth and Planetary Astrophysics', 'Solar an...",NaN,NaN,[],2025-12-23,Science white paper submitted in the context o...,NaN,0,"[None, None, None, None, None, None, None, Non...",0,0,0
3,Diffusive or Ballistic? Distributions and Spec...,In the standard Galactic cosmic-ray (CR) parad...,"['Yutaka Fujita', 'Rohta Takahashi', 'Norita K...",3.0,6.0,0.0,arxiv.org/pdf/2512.17994,High Energy Astrophysical Phenomena,NaN,NaN,NaN,['High energy astrophysics (739) — Cosmic rays...,2025-12-23,"6 pages, 3 figures",NaN,0,"[None, None, None]",0,0,0
4,Pulsational Instability of Quasi-Stars: Interp...,"The JWST discovery of ""Little Red Dots"" (LRDs)...","['Matteo Cantiello', 'Jake B. Hassan', 'Rosalb...",6.0,11.0,0.0,arxiv.org/pdf/2512.17997,High Energy Astrophysical Phenomena,"['Astrophysics of Galaxies', 'Solar and Stella...",NaN,NaN,['Early Universe — Supermassive black holes — ...,2025-12-23,"11 pages, 6 figures, submitted to ApJL",NaN,0,"[None, None, None, None, None, None, None, None]",0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,Cosmic Himalayas in CROCODILE : Probing the Ex...,The recently reported Cosmic Himalayas (CH) --...,"['Yuto Kuwayama', 'Yongming Liang', 'Kentaro N...",10.0,18.0,5.0,arxiv.org/pdf/2512.24966,Cosmology and Nongalactic Astrophysics,['Astrophysics of Galaxies'],NaN,NaN,['Hydrodynamicalsimulations(767)'],2025-12-31,"18 pages, 10 figures, submitted to ApJ",NaN,0,"[None, None, None, None, None, None, None, Non...",0,0,0
323,Multi-Frequency Study of FRB20201124A with the...,We present results from multi-epoch observatio...,"['C. Dudeja', 'J. Roy', 'U. Panda', 'S. Bhatta...",11.0,17.0,1.0,arxiv.org/pdf/2512.24978,High Energy Astrophysical Phenomena,NaN,NaN,NaN,[],2025-12-31,NaN,NaN,0,"[None, None, None, None]",0,0,0
324,Distributions of wide binary stars in theory a...,"Using the censored catalog of 103,169 resolved...",['Valeri V. Makarov'],7.0,13.0,1.0,arxiv.org/pdf/2512.25002,Solar and Stellar Astrophysics,NaN,NaN,NaN,[],2025-12-31,To be published in Astron J,NaN,0,[None],0,0,1
325,Towards precision cosmology with Voids x CMB c...,We construct and validate a set of multi-purpo...,"['Mar Pérez Sar', 'Carlos Hernández Monteagudo...",8.0,19.0,1.0,arxiv.org/pdf/2512.25040,Cosmology and Nongalactic Astrophysics,NaN,NaN,NaN,['Cosmology: Large-scale structure - Cosmic mi...,2025-12-31,"19 pages, 8 figures, submitted to A&A",NaN,0,"[None, None, None, None]",0,0,0


In [29]:
ads_journals = pd.read_csv('/Users/ainsleylewis/Documents/Astronomy/arXiver/papers_with_journals_4.csv')

In [32]:
# Adding in ADS Journals to journals column if missing
for i in range(len(merged_data)):
    if isinstance(merged_data.at[i, 'journals'], str):
        continue
    else:
        title = merged_data.at[i, 'title']
        matched_row = ads_journals[ads_journals['title'] == title]
        if not matched_row.empty:
            ads_journal = matched_row.iloc[0]['abs_journal']
            if isinstance(ads_journal, str) and ads_journal.strip() != '':
                merged_data.at[i, 'journals'] = ads_journal

# Set all rows with 'arxiv e-prints' to NaN in journals column
counter = 0
for i in range(len(merged_data)):
    if isinstance(merged_data.at[i, 'journals'], str):
        if merged_data.at[i, 'journals'] == 'arXiv e-prints':
            merged_data.at[i, 'journals'] = np.nan
            counter += 1

print(f"Total papers updated from 'arXiv e-prints' to NaN in journals column: {counter}")

Total papers updated from 'arXiv e-prints' to NaN in journals column: 314


In [33]:
merged_data

,title,abstract,authors,figures,pages,tables,pdf_link,primary_subject,secondary_subjects,submitted_journal,published_journal,keywords,date,comments,journals,num_checks,affiliations,total_citations,non_self_citations,journal_flag
0,A Correlation Between Black Hole Mass and Dark...,We report the discovery of a positive correlat...,['John K. Nino'],1.0,12.0,2.0,arxiv.org/pdf/2512.17964,Astrophysics of Galaxies,['Cosmology and Nongalactic Astrophysics'],NaN,NaN,and total stellar mass—has been established fo...,2025-12-23,"6 pages, 1 figure, 2 tables. v2: Corrected Li ...",NaN,0,[None],0,0,0
1,CoBiTS: Single-detector discrimination of bina...,"We develop a Conformer neural network, called ...","['Matthew VanDyke', 'Kexuan Wu', 'Sukanta Bose']",13.0,22.0,0.0,arxiv.org/pdf/2512.17975,Instrumentation and Methods for Astrophysics,"['High Energy Astrophysical Phenomena', 'Gener...",NaN,NaN,[],2025-12-23,"22 pages, 13 figures",NaN,0,"[None, None, None]",0,0,0
2,Bridging stellar evolution and planet formatio...,"Stars and planets form, live, and evolve in un...","['Akke Corporaal', 'Toon De Prins', 'Léa Planq...",0.0,4.0,0.0,arxiv.org/pdf/2512.17976,Instrumentation and Methods for Astrophysics,"['Earth and Planetary Astrophysics', 'Solar an...",NaN,NaN,[],2025-12-23,Science white paper submitted in the context o...,NaN,0,"[None, None, None, None, None, None, None, Non...",0,0,0
3,Diffusive or Ballistic? Distributions and Spec...,In the standard Galactic cosmic-ray (CR) parad...,"['Yutaka Fujita', 'Rohta Takahashi', 'Norita K...",3.0,6.0,0.0,arxiv.org/pdf/2512.17994,High Energy Astrophysical Phenomena,NaN,NaN,NaN,['High energy astrophysics (739) — Cosmic rays...,2025-12-23,"6 pages, 3 figures",NaN,0,"[None, None, None]",0,0,0
4,Pulsational Instability of Quasi-Stars: Interp...,"The JWST discovery of ""Little Red Dots"" (LRDs)...","['Matteo Cantiello', 'Jake B. Hassan', 'Rosalb...",6.0,11.0,0.0,arxiv.org/pdf/2512.17997,High Energy Astrophysical Phenomena,"['Astrophysics of Galaxies', 'Solar and Stella...",NaN,NaN,['Early Universe — Supermassive black holes — ...,2025-12-23,"11 pages, 6 figures, submitted to ApJL",NaN,0,"[None, None, None, None, None, None, None, None]",0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,Cosmic Himalayas in CROCODILE : Probing the Ex...,The recently reported Cosmic Himalayas (CH) --...,"['Yuto Kuwayama', 'Yongming Liang', 'Kentaro N...",10.0,18.0,5.0,arxiv.org/pdf/2512.24966,Cosmology and Nongalactic Astrophysics,['Astrophysics of Galaxies'],NaN,NaN,['Hydrodynamicalsimulations(767)'],2025-12-31,"18 pages, 10 figures, submitted to ApJ",NaN,0,"[None, None, None, None, None, None, None, Non...",0,0,0
323,Multi-Frequency Study of FRB20201124A with the...,We present results from multi-epoch observatio...,"['C. Dudeja', 'J. Roy', 'U. Panda', 'S. Bhatta...",11.0,17.0,1.0,arxiv.org/pdf/2512.24978,High Energy Astrophysical Phenomena,NaN,NaN,NaN,[],2025-12-31,NaN,NaN,0,"[None, None, None, None]",0,0,0
324,Distributions of wide binary stars in theory a...,"Using the censored catalog of 103,169 resolved...",['Valeri V. Makarov'],7.0,13.0,1.0,arxiv.org/pdf/2512.25002,Solar and Stellar Astrophysics,NaN,NaN,NaN,[],2025-12-31,To be published in Astron J,NaN,0,[None],0,0,1
325,Towards precision cosmology with Voids x CMB c...,We construct and validate a set of multi-purpo...,"['Mar Pérez Sar', 'Carlos Hernández Monteagudo...",8.0,19.0,1.0,arxiv.org/pdf/2512.25040,Cosmology and Nongalactic Astrophysics,NaN,NaN,NaN,['Cosmology: Large-scale structure - Cosmic mi...,2025-12-31,"19 pages, 8 figures, submitted to A&A",NaN,0,"[None, None, None, None]",0,0,0


In [34]:
merged_data['journal_flag'] = merged_data['published_journal'].notnull().astype(int)
merged_data

# Update journal_flag based on comments
for i in range(len(merged_data)):
    if merged_data['journal_flag'][i] == 1:
        continue
    elif isinstance(merged_data.at[i, 'comments'], str):
        comments_lower = merged_data.at[i, 'comments'].lower()
        if 'published' in comments_lower or 'accepted' in comments_lower:
            merged_data.at[i, 'journal_flag'] = 1

# Update journal_flag based on journals column from submitted_journals.py
for i in range(len(merged_data)):
    if merged_data['journal_flag'][i] == 1:
        continue
    elif isinstance(merged_data.at[i, 'journals'], str):
        merged_data.at[i, 'journal_flag'] = 1

In [35]:
# Count number of times the word 'submitted' appears in comments
counter = 0
for i in range(len(merged_data)):
    if isinstance(merged_data.at[i, 'comments'], str):
        comments_lower = merged_data.at[i, 'comments'].lower()
        if 'submitted' in comments_lower:
            print(f"'Submitted' found in comments for index {i}: {merged_data.at[i, 'comments']}")
            counter += 1
print(f"Total papers with 'submitted' in comments: {counter}")

'Submitted' found in comments for index 2: Science white paper submitted in the context of ESO's Expanding Horizons call
'Submitted' found in comments for index 4: 11 pages, 6 figures, submitted to ApJL
'Submitted' found in comments for index 8: Submitted to ApJ, 18 pages, 15 figures
'Submitted' found in comments for index 10: submitted to A&A
'Submitted' found in comments for index 12: Submitted to the ESO Expanding Horizons initiative and posted on arXiv to encourage community discussion. Comments are welcome via email; voluntary endorsement of the general principles may be expressed via this google form: this https URL
'Submitted' found in comments for index 15: 15 pages, 10 figures; submitted to A&A
'Submitted' found in comments for index 18: Submitted to AAS Journals
'Submitted' found in comments for index 19: Submitted in response to the call for White Papers for the definition of the ESO next transformational facility. this https URL. 6 pages
'Submitted' found in comments for in

In [36]:
# Count the number of times the words 'submitted' or 'journal' or 'published' or 'accepted' appears in comments
counter = 0
for i in merged_data.index:
    if merged_data['published_journal'][i] is not None and isinstance(merged_data['published_journal'][i], str):
        continue
    if isinstance(merged_data.at[i, 'comments'], str):
        comments_lower = merged_data.at[i, 'comments'].lower()
        if 'published' in comments_lower or 'accepted' in comments_lower:
            counter += 1

print(f"Total papers with publication info in comments: {counter}")

Total papers with publication info in comments: 88


In [37]:
# Percentage of Journals from journal_flag
journal_counts = merged_data['journal_flag'].value_counts(normalize=True) * 100
journal_counts

print("Percentage of papers with published journals:", journal_counts.get(1, 0))

merged_data['journal_flag_with_submitted'] = merged_data['journal_flag']
# Assume all submitted journals are published
for i in range(len(merged_data)):
    if merged_data['journal_flag'][i] == 0:
        if 'submitted' in str(merged_data.at[i, 'comments']).lower():
            merged_data.at[i, 'journal_flag_with_submitted'] = 1

# Percentage of Journals from journal_flag_with_submitted
journal_counts_with_submitted = merged_data['journal_flag_with_submitted'].value_counts(normalize=True) * 100
journal_counts_with_submitted

print("Percentage of papers with published journals (including submitted):", journal_counts_with_submitted.get(1, 0))

Percentage of papers with published journals: 33.02752293577982
Percentage of papers with published journals (including submitted): 51.07033639143731


In [38]:
# Missmatch between journal_flags and citations
mismatch_count = 0

for i in range(len(merged_data)):
    if merged_data['journal_flag'][i] == 0 and merged_data['total_citations'][i] > 0:
        print(f"Mismatch found for paper - {merged_data['title'][i]} : journal_flag = {merged_data['journal_flag'][i]}, total_citations = {merged_data['total_citations'][i]}")
        mismatch_count += 1

print(f"Total mismatches between journal_flag and citations: {mismatch_count}")

Mismatch found for paper - Monitoring and Evaluating Astronomy-for-Development Initiatives : journal_flag = 0, total_citations = 2
Mismatch found for paper - Constrained Brownian-bridge prior for neutron-star equation-of-state inference : journal_flag = 0, total_citations = 1
Mismatch found for paper - GW231123: A Case for Binary Microlensing in a Strong Lensing Field : journal_flag = 0, total_citations = 1
Mismatch found for paper - Are Primordial Black Holes Truly Fine-Tuned? : journal_flag = 0, total_citations = 1
Mismatch found for paper - SN 2025ogs: A Spectroscopically-Normal Type Ia Supernova at z = 2 as a Benchmark for Redshift Evolution : journal_flag = 0, total_citations = 1
Mismatch found for paper - Observational constraints on early time non-phantom behaviour of dynamical dark energy : journal_flag = 0, total_citations = 1
Mismatch found for paper - Decay of $f(R)$ quintessence into dark matter: mitigating the Hubble tension? : journal_flag = 0, total_citations = 1
Mismatc

In [39]:
merged_data

,title,abstract,authors,figures,pages,tables,pdf_link,primary_subject,secondary_subjects,submitted_journal,...,keywords,date,comments,journals,num_checks,affiliations,total_citations,non_self_citations,journal_flag,journal_flag_with_submitted
0,A Correlation Between Black Hole Mass and Dark...,We report the discovery of a positive correlat...,['John K. Nino'],1.0,12.0,2.0,arxiv.org/pdf/2512.17964,Astrophysics of Galaxies,['Cosmology and Nongalactic Astrophysics'],NaN,...,and total stellar mass—has been established fo...,2025-12-23,"6 pages, 1 figure, 2 tables. v2: Corrected Li ...",NaN,0,[None],0,0,0,0
1,CoBiTS: Single-detector discrimination of bina...,"We develop a Conformer neural network, called ...","['Matthew VanDyke', 'Kexuan Wu', 'Sukanta Bose']",13.0,22.0,0.0,arxiv.org/pdf/2512.17975,Instrumentation and Methods for Astrophysics,"['High Energy Astrophysical Phenomena', 'Gener...",NaN,...,[],2025-12-23,"22 pages, 13 figures",NaN,0,"[None, None, None]",0,0,0,0
2,Bridging stellar evolution and planet formatio...,"Stars and planets form, live, and evolve in un...","['Akke Corporaal', 'Toon De Prins', 'Léa Planq...",0.0,4.0,0.0,arxiv.org/pdf/2512.17976,Instrumentation and Methods for Astrophysics,"['Earth and Planetary Astrophysics', 'Solar an...",NaN,...,[],2025-12-23,Science white paper submitted in the context o...,NaN,0,"[None, None, None, None, None, None, None, Non...",0,0,0,1
3,Diffusive or Ballistic? Distributions and Spec...,In the standard Galactic cosmic-ray (CR) parad...,"['Yutaka Fujita', 'Rohta Takahashi', 'Norita K...",3.0,6.0,0.0,arxiv.org/pdf/2512.17994,High Energy Astrophysical Phenomena,NaN,NaN,...,['High energy astrophysics (739) — Cosmic rays...,2025-12-23,"6 pages, 3 figures",NaN,0,"[None, None, None]",0,0,0,0
4,Pulsational Instability of Quasi-Stars: Interp...,"The JWST discovery of ""Little Red Dots"" (LRDs)...","['Matteo Cantiello', 'Jake B. Hassan', 'Rosalb...",6.0,11.0,0.0,arxiv.org/pdf/2512.17997,High Energy Astrophysical Phenomena,"['Astrophysics of Galaxies', 'Solar and Stella...",NaN,...,['Early Universe — Supermassive black holes — ...,2025-12-23,"11 pages, 6 figures, submitted to ApJL",NaN,0,"[None, None, None, None, None, None, None, None]",0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,Cosmic Himalayas in CROCODILE : Probing the Ex...,The recently reported Cosmic Himalayas (CH) --...,"['Yuto Kuwayama', 'Yongming Liang', 'Kentaro N...",10.0,18.0,5.0,arxiv.org/pdf/2512.24966,Cosmology and Nongalactic Astrophysics,['Astrophysics of Galaxies'],NaN,...,['Hydrodynamicalsimulations(767)'],2025-12-31,"18 pages, 10 figures, submitted to ApJ",NaN,0,"[None, None, None, None, None, None, None, Non...",0,0,0,1
323,Multi-Frequency Study of FRB20201124A with the...,We present results from multi-epoch observatio...,"['C. Dudeja', 'J. Roy', 'U. Panda', 'S. Bhatta...",11.0,17.0,1.0,arxiv.org/pdf/2512.24978,High Energy Astrophysical Phenomena,NaN,NaN,...,[],2025-12-31,NaN,NaN,0,"[None, None, None, None]",0,0,0,0
324,Distributions of wide binary stars in theory a...,"Using the censored catalog of 103,169 resolved...",['Valeri V. Makarov'],7.0,13.0,1.0,arxiv.org/pdf/2512.25002,Solar and Stellar Astrophysics,NaN,NaN,...,[],2025-12-31,To be published in Astron J,NaN,0,[None],0,0,1,1
325,Towards precision cosmology with Voids x CMB c...,We construct and validate a set of multi-purpo...,"['Mar Pérez Sar', 'Carlos Hernández Monteagudo...",8.0,19.0,1.0,arxiv.org/pdf/2512.25040,Cosmology and Nongalactic Astrophysics,NaN,NaN,...,['Cosmology: Large-scale structure - Cosmic mi...,2025-12-31,"19 pages, 8 figures, submitted to A&A",NaN,0,"[None, None, None, None]",0,0,0,1


In [42]:
# List of authors with None Affiliation
none_affil_authors = []
for i in range(len(merged_data)):
    affils = merged_data.at[i, 'affiliations']
    
    # Parse authors string to list if necessary
    authors_val = merged_data.at[i, 'authors']
    if isinstance(authors_val, str):
        try:
            authors = ast.literal_eval(authors_val)
        except:
            authors = []
    else:
        authors = authors_val

    if isinstance(affils, list) and isinstance(authors, list):
        for author, affil in zip(authors, affils):
            if affil is None:
                none_affil_authors.append(author)

print(f"Total authors with None affiliation: {len(none_affil_authors)}")
none_affil_authors

Total authors with None affiliation: 2806


['John K. Nino',
 'Matthew VanDyke',
 'Kexuan Wu',
 'Sukanta Bose',
 'Akke Corporaal',
 'Toon De Prins',
 'Léa Planquart',
 'Kateryna Andrych',
 'Narsireddy Anugu',
 'Devika Kamath',
 'Jens Kammerer',
 'Stefan Kraus',
 'Fonteini Lykou',
 'Alexis Matter',
 'Claudia Paladini',
 'Marie M. Rodríguez S.',
 'Hans Van Winckel',
 'Yutaka Fujita',
 'Rohta Takahashi',
 'Norita Kawanaka',
 'Matteo Cantiello',
 'Jake B. Hassan',
 'Rosalba Perna',
 'Philip J. Armitage',
 'Mitchell C. Begelman',
 'Yan-Fei Jiang',
 'Taeho Ryu',
 'Richard H. D. Townsend',
 'Maria Ramos',
 'Timothy Cohen',
 'Mariangela Lisanti',
 'M. Grant Roberts',
 'Aarna Garg',
 'Tesla Jeltema',
 'Stefano Profumo',
 'A. Damonte',
 'I. Pillitteri',
 'A. Maggio',
 'A. García Muñoz',
 'G. Micela',
 'Alison L. Coil',
 'David S. N. Rupke',
 'Serena Perrotta',
 'Saloni Agrawal',
 'Cassandra Lochhaas',
 'Joyful E. Mdhluli',
 'J. de la Cruz Rodríguez',
 'G. B. Scharmer',
 'P. Sütterlin',
 'J. Leenaarts',
 'M. G. Löfdahl',
 'D. Kiselman',
 '

In [44]:
# Save none_affil_authors to a text file
with open('/Users/ainsleylewis/Documents/Astronomy/arXiver/none_affil_authors.txt', 'w') as f:
    for author in none_affil_authors:
        f.write(f"{author}\n")

In [48]:
# Adding ADS filled affiliations
ads_affils = pd.read_csv('/Users/ainsleylewis/Documents/Astronomy/arXiver/missed_affils_1.csv')

ads_affils

,Author,Affiliation
0,John K. Nino,Not Found
1,Matthew VanDyke,Not Found
2,Kexuan Wu,Not Found
3,Sukanta Bose,Not Found
4,Akke Corporaal,Not Found
...,...,...
2801,Peter McGill,Not Found
2802,Armin Rest,Not Found
2803,César Rojas-Bravo,Not Found
2804,Melissa Shahbandeh,Not Found


In [49]:
# Adding affils from missing affils 
for i in range(len(merged_data)):
    authors_val = merged_data.at[i, 'authors']
    if isinstance(authors_val, str):
        try:
            authors = ast.literal_eval(authors_val)
        except:
            authors = []
    else:
        authors = authors_val

    affils = merged_data.at[i, 'affiliations']
    if not isinstance(affils, list):
        affils = [None] * len(authors)

    for j, author in enumerate(authors):
        if affils[j] is None:
            matched_row = ads_affils[ads_affils['author'] == author]
            if not matched_row.empty:
                ads_affil = matched_row.iloc[0]['affiliation']
                if isinstance(ads_affil, str) and ads_affil.strip() != '':
                    affils[j] = ads_affil

    merged_data.at[i, 'affiliations'] = affils

KeyError: 'author'

In [4]:
merged_data = pd.read_csv('/Users/rommulus/Documents/Astronomy/arXiver/2025_Data_missing.csv')

In [5]:
# Percentage of Journals from journal_flag
journal_counts = merged_data['journal_flag'].value_counts(normalize=True) * 100
journal_counts

print("Percentage of papers with published journals:", journal_counts.get(1, 0))

merged_data['journal_flag_with_submitted'] = merged_data['journal_flag']
# Assume all submitted journals are published
for i in range(len(merged_data)):
    if merged_data['journal_flag'][i] == 0:
        if 'submitted' in str(merged_data.at[i, 'comments']).lower():
            merged_data.at[i, 'journal_flag_with_submitted'] = 1

# Percentage of Journals from journal_flag_with_submitted
journal_counts_with_submitted = merged_data['journal_flag_with_submitted'].value_counts(normalize=True) * 100
journal_counts_with_submitted

print("Percentage of papers with published journals (including submitted):", journal_counts_with_submitted.get(1, 0))

Percentage of papers with published journals: 32.11009174311927
Percentage of papers with published journals (including submitted): 50.45871559633027


In [6]:
merged_data

,title,abstract,authors,figures,pages,tables,pdf_link,primary_subject,secondary_subjects,submitted_journal,...,keywords,date,comments,journals,num_checks,affiliations,total_citations,non_self_citations,journal_flag,journal_flag_with_submitted
0,A Correlation Between Black Hole Mass and Dark...,We report the discovery of a positive correlat...,['John K. Nino'],1.0,12.0,2.0,arxiv.org/pdf/2512.17964,Astrophysics of Galaxies,['Cosmology and Nongalactic Astrophysics'],NaN,...,and total stellar mass—has been established fo...,2025-12-23,"6 pages, 1 figure, 2 tables. v2: Corrected Li ...",NaN,0,[None],0,0,0,0
1,CoBiTS: Single-detector discrimination of bina...,"We develop a Conformer neural network, called ...","['Matthew VanDyke', 'Kexuan Wu', 'Sukanta Bose']",13.0,22.0,0.0,arxiv.org/pdf/2512.17975,Instrumentation and Methods for Astrophysics,"['High Energy Astrophysical Phenomena', 'Gener...",NaN,...,[],2025-12-23,"22 pages, 13 figures",NaN,0,"[None, None, None]",0,0,0,0
2,Bridging stellar evolution and planet formatio...,"Stars and planets form, live, and evolve in un...","['Akke Corporaal', 'Toon De Prins', 'Léa Planq...",0.0,4.0,0.0,arxiv.org/pdf/2512.17976,Instrumentation and Methods for Astrophysics,"['Earth and Planetary Astrophysics', 'Solar an...",NaN,...,[],2025-12-23,Science white paper submitted in the context o...,NaN,0,"[None, None, None, None, None, None, None, Non...",0,0,0,1
3,Diffusive or Ballistic? Distributions and Spec...,In the standard Galactic cosmic-ray (CR) parad...,"['Yutaka Fujita', 'Rohta Takahashi', 'Norita K...",3.0,6.0,0.0,arxiv.org/pdf/2512.17994,High Energy Astrophysical Phenomena,NaN,NaN,...,['High energy astrophysics (739) — Cosmic rays...,2025-12-23,"6 pages, 3 figures",NaN,0,"[None, None, None]",0,0,0,0
4,Pulsational Instability of Quasi-Stars: Interp...,"The JWST discovery of ""Little Red Dots"" (LRDs)...","['Matteo Cantiello', 'Jake B. Hassan', 'Rosalb...",6.0,11.0,0.0,arxiv.org/pdf/2512.17997,High Energy Astrophysical Phenomena,"['Astrophysics of Galaxies', 'Solar and Stella...",NaN,...,['Early Universe — Supermassive black holes — ...,2025-12-23,"11 pages, 6 figures, submitted to ApJL",NaN,0,"[None, None, None, None, None, None, None, None]",0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,Cosmic Himalayas in CROCODILE : Probing the Ex...,The recently reported Cosmic Himalayas (CH) --...,"['Yuto Kuwayama', 'Yongming Liang', 'Kentaro N...",10.0,18.0,5.0,arxiv.org/pdf/2512.24966,Cosmology and Nongalactic Astrophysics,['Astrophysics of Galaxies'],NaN,...,['Hydrodynamicalsimulations(767)'],2025-12-31,"18 pages, 10 figures, submitted to ApJ",NaN,0,"[None, None, None, None, None, None, None, Non...",0,0,0,1
323,Multi-Frequency Study of FRB20201124A with the...,We present results from multi-epoch observatio...,"['C. Dudeja', 'J. Roy', 'U. Panda', 'S. Bhatta...",11.0,17.0,1.0,arxiv.org/pdf/2512.24978,High Energy Astrophysical Phenomena,NaN,NaN,...,[],2025-12-31,NaN,NaN,0,"[None, None, None, None]",0,0,0,0
324,Distributions of wide binary stars in theory a...,"Using the censored catalog of 103,169 resolved...",['Valeri V. Makarov'],7.0,13.0,1.0,arxiv.org/pdf/2512.25002,Solar and Stellar Astrophysics,NaN,NaN,...,[],2025-12-31,To be published in Astron J,NaN,0,[None],0,0,1,1
325,Towards precision cosmology with Voids x CMB c...,We construct and validate a set of multi-purpo...,"['Mar Pérez Sar', 'Carlos Hernández Monteagudo...",8.0,19.0,1.0,arxiv.org/pdf/2512.25040,Cosmology and Nongalactic Astrophysics,NaN,NaN,...,['Cosmology: Large-scale structure - Cosmic mi...,2025-12-31,"19 pages, 8 figures, submitted to A&A",NaN,0,"[None, None, None, None]",0,0,0,1
